**LIBRARIES AND SETUP**

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
import time
import warnings
from datetime import datetime, timedelta
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import json
from collections import defaultdict, deque
import threading
import queue
import random

# Suppress warnings
warnings.filterwarnings('ignore')

# Initial print statements
print("CICIDS2017 Random Forest Real-Time Evaluation Pipeline")
print("=" * 60)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

CICIDS2017 Random Forest Real-Time Evaluation Pipeline
Start time: 2025-05-29 23:18:02


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**CONFIGURATION**

This cell defines the configuration parameters for the evaluation system, including file paths for preprocessed data, models, and output directories. It creates a timestamped output directory structure and sets real-time processing parameters like batch size and alert thresholds.

In [3]:
RANDOM_STATE = 42
# Input paths
PREPROCESSED_DATA_PATH = '/content/drive/MyDrive/furssah/preprocessing_output/'
PREPROCESSED_MODELS_PATH = os.path.join(PREPROCESSED_DATA_PATH, 'models')
PREPROCESSED_FEATURES_PATH = os.path.join(PREPROCESSED_DATA_PATH, 'features')
TRAINING_OUTPUT_BASE = '/content/drive/MyDrive/furssah/training_output/'
TRAINING_MODELS_PATH = os.path.join(TRAINING_OUTPUT_BASE, 'trained_models')

# Output paths with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
EVALUATION_OUTPUT_BASE = f'/content/drive/MyDrive/furssah/evaluation_output/'

# Create directory structure
EVALUATION_PATHS = {
    'results': os.path.join(EVALUATION_OUTPUT_BASE, 'results'),
    'visualizations': os.path.join(EVALUATION_OUTPUT_BASE, 'visualizations'),
    'logs': os.path.join(EVALUATION_OUTPUT_BASE, 'logs'),
    'reports': os.path.join(EVALUATION_OUTPUT_BASE, 'reports')
}

for path in EVALUATION_PATHS.values():
    os.makedirs(path, exist_ok=True)

# Real-time configuration
REALTIME_CONFIG = {
    'batch_size': 100,           # Process flows in batches
    'alert_threshold': 0.8,      # Confidence threshold for alerts
    'window_size': 300,          # 5-minute sliding window
    'max_queue_size': 1000,      # Maximum queue size
    'processing_interval': 1.0,   # Process every 1 second
}

**System Diagnostics**

This cell checks the availability of required file paths and directories, ensuring that the preprocessed data, models, and output directories exist. It prints a summary of the system status, listing files found or reporting errors if paths are missing.

In [4]:
print("\n" + "="*50)
print("SYSTEM DIAGNOSTICS")
print("="*50)

def check_system_requirements():
    """Check if all required files and directories exist"""
    diagnostics = {
        'preprocessed_models': PREPROCESSED_MODELS_PATH,
        'preprocessed_features': PREPROCESSED_FEATURES_PATH,
        'training_models': TRAINING_MODELS_PATH,
        'evaluation_output': EVALUATION_OUTPUT_BASE
    }

    print("Checking file paths...")
    system_status = {}

    for component, path in diagnostics.items():
        try:
            if os.path.exists(path):
                files = os.listdir(path)
                system_status[component] = {'status': 'OK', 'files': files}
                print(f"✓ {component}: {len(files)} files found")
                for file in files[:3]:  # Show first 3 files
                    print(f"  - {file}")
                if len(files) > 3:
                    print(f"  ... and {len(files)-3} more files")
            else:
                system_status[component] = {'status': 'MISSING', 'files': []}
                print(f"  {component}: Directory not found")
        except Exception as e:
            system_status[component] = {'status': 'ERROR', 'error': str(e)}
            print(f"  {component}: Error - {e}")

    return system_status

system_status = check_system_requirements()


SYSTEM DIAGNOSTICS
Checking file paths...
✓ preprocessed_models: 2 files found
  - scaler.joblib
  - label_encoders.joblib
✓ preprocessed_features: 1 files found
  - feature_names.txt
✓ training_models: 3 files found
  - attack_type_random_forest_model.joblib
  - traffic_source_random_forest_model.joblib
  - operator_type_random_forest_model.joblib
✓ evaluation_output: 4 files found
  - results
  - visualizations
  - logs
  ... and 1 more files


**Load Models and Preprocessing Components**

This cell loads pretrained models, feature names, scaler, and label encoders from specified paths. It uses a `ModelLoader` class to handle loading with fallback options if files are missing, and prints the loading status and time taken.

In [5]:
print("\n" + "="*50)
print("LOADING MODELS AND PREPROCESSING COMPONENTS")
print("="*50)

start_load_time = time.time()

# Initialize components
feature_names = []
scaler = None
label_encoders = {}
models = {}
model_metadata = {}

class ModelLoader:
    def __init__(self):
        self.load_status = {}

    def load_feature_names(self):
        """Load feature names from file"""
        try:
            feature_names_path = os.path.join(PREPROCESSED_FEATURES_PATH, 'feature_names.txt')
            if os.path.exists(feature_names_path):
                with open(feature_names_path, 'r') as f:
                    features = [line.strip() for line in f.readlines() if line.strip()]
                self.load_status['features'] = f"✓ Loaded {len(features)} features"
                return features
            else:
                # Fallback feature names
                fallback_features = [
                    'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
                    'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
                    'Fwd Packet Length Mean', 'Bwd Packet Length Mean',
                    'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean'
                ]
                self.load_status['features'] = f"  Using {len(fallback_features)} fallback features"
                return fallback_features
        except Exception as e:
            self.load_status['features'] = f"  Error loading features: {e}"
            return []

    def load_scaler(self):
        """Load feature scaler"""
        try:
            scaler_path = os.path.join(PREPROCESSED_MODELS_PATH, 'scaler.joblib')
            if os.path.exists(scaler_path):
                scaler = joblib.load(scaler_path)
                self.load_status['scaler'] = "✓ Loaded StandardScaler"
                return scaler
            else:
                self.load_status['scaler'] = "  Scaler not found, creating default"
                return StandardScaler()
        except Exception as e:
            self.load_status['scaler'] = f"  Error loading scaler: {e}"
            return None

    def load_label_encoders(self):
        """Load label encoders"""
        try:
            encoders_path = os.path.join(PREPROCESSED_MODELS_PATH, 'label_encoders.joblib')
            if os.path.exists(encoders_path):
                encoders = joblib.load(encoders_path)
                self.load_status['encoders'] = f"✓ Loaded {len(encoders)} label encoders"
                return encoders
            else:
                self.load_status['encoders'] = "  Label encoders not found"
                return {}
        except Exception as e:
            self.load_status['encoders'] = f"  Error loading encoders: {e}"
            return {}

    def load_models(self):
        """Load trained models"""
        models = {}
        metadata = {}
        tasks = ['attack_type', 'traffic_source', 'operator_type']

        for task in tasks:
            try:
                model_path = os.path.join(TRAINING_MODELS_PATH, f"{task}_random_forest_model.joblib")
                if os.path.exists(model_path):
                    model = joblib.load(model_path)
                    models[task] = model

                    # Get model metadata
                    if hasattr(model, 'n_estimators'):
                        metadata[task] = {
                            'n_estimators': model.n_estimators,
                            'max_depth': getattr(model, 'max_depth', 'None'),
                            'n_features': getattr(model, 'n_features_in_', 'Unknown')
                        }

                    self.load_status[f'model_{task}'] = f"✓ Loaded {task} model"
                else:
                    self.load_status[f'model_{task}'] = f"⚠️ {task} model not found"
            except Exception as e:
                self.load_status[f'model_{task}'] = f"❌ Error loading {task}: {e}"

        return models, metadata

# Load all components
loader = ModelLoader()
feature_names = loader.load_feature_names()
scaler = loader.load_scaler()
label_encoders = loader.load_label_encoders()
models, model_metadata = loader.load_models()

# Print loading status
print("Loading Status:")
for component, status in loader.load_status.items():
    print(f"  {status}")

load_time = time.time() - start_load_time
print(f"\nModel loading completed in {load_time:.2f} seconds")


LOADING MODELS AND PREPROCESSING COMPONENTS
Loading Status:
  ✓ Loaded 7 features
  ✓ Loaded StandardScaler
  ✓ Loaded 3 label encoders
  ✓ Loaded attack_type model
  ✓ Loaded traffic_source model
  ✓ Loaded operator_type model

Model loading completed in 3.43 seconds


REAL-TIME TRAFFIC ANALYZER CLASS

This cell defines the `RealTimeTrafficAnalyzer` class, which handles real-time network traffic analysis. It preprocesses network flows, makes predictions using loaded models, generates alerts based on confidence thresholds, and tracks statistics. The analyzer is initialized if models are successfully loaded.

In [6]:
print("\n" + "="*50)
print("INITIALIZING REAL-TIME ANALYZER")
print("="*50)

class RealTimeTrafficAnalyzer:
    def __init__(self, models, scaler, label_encoders, feature_names, config):
        self.models = models
        self.scaler = scaler
        self.label_encoders = label_encoders
        self.feature_names = feature_names
        self.config = config

        # Real-time data structures
        self.traffic_queue = queue.Queue(maxsize=config['max_queue_size'])
        self.results_history = deque(maxlen=1000)
        self.alert_history = deque(maxlen=100)
        self.statistics = defaultdict(int)

        # Timing and performance
        self.start_time = time.time()
        self.processed_flows = 0
        self.alerts_generated = 0

        # Thread control
        self.is_running = False
        self.processing_thread = None

        print(f"✓ Real-time analyzer initialized")
        print(f"  - Models: {list(self.models.keys())}")
        print(f"  - Features: {len(self.feature_names)}")
        print(f"  - Queue size: {self.config['max_queue_size']}")

    def preprocess_sample(self, flow_data):
        """Preprocess a single network flow sample"""
        try:
            # Convert to DataFrame
            if isinstance(flow_data, dict):
                df = pd.DataFrame([flow_data])
            else:
                df = pd.DataFrame(flow_data)

            # Add missing features with default values
            for feature in self.feature_names:
                if feature not in df.columns:
                    df[feature] = 0.0

            # Select and order features
            df = df[self.feature_names]

            # Handle infinite and NaN values
            df = df.replace([np.inf, -np.inf], np.nan)
            df = df.fillna(0.0)

            # Scale features if scaler is available
            if self.scaler is not None:
                scaled_data = self.scaler.transform(df)
                return scaled_data
            else:
                return df.values

        except Exception as e:
            print(f"Preprocessing error: {e}")
            # Return zeros if preprocessing fails
            return np.zeros((1, len(self.feature_names)))

    def predict_flow(self, flow_data):
        """Make predictions for a single flow"""
        try:
            # Preprocess the flow
            preprocessed = self.preprocess_sample(flow_data)

            # Make predictions with all models
            predictions = {}
            confidences = {}

            for task_name, model in self.models.items():
                try:
                    # Get prediction and probability
                    pred = model.predict(preprocessed)[0]
                    pred_proba = model.predict_proba(preprocessed)[0]

                    # Decode prediction if encoder exists
                    if task_name in self.label_encoders:
                        try:
                            decoded_pred = self.label_encoders[task_name].inverse_transform([pred])[0]
                        except:
                            decoded_pred = str(pred)
                    else:
                        decoded_pred = str(pred)

                    predictions[task_name] = decoded_pred
                    confidences[task_name] = float(np.max(pred_proba))

                except Exception as e:
                    print(f"Prediction error for {task_name}: {e}")
                    predictions[task_name] = "UNKNOWN"
                    confidences[task_name] = 0.0

            return predictions, confidences

        except Exception as e:
            print(f"Flow prediction error: {e}")
            return {}, {}

    def generate_alert(self, flow_data, predictions, confidences):
        """Generate security alert if threat detected"""
        alert = {
            'timestamp': datetime.now().isoformat(),
            'flow_id': len(self.results_history),
            'predictions': predictions,
            'confidences': confidences,
            'alert_level': 'LOW',
            'threat_detected': False
        }

        # Determine alert level
        max_confidence = max(confidences.values()) if confidences else 0

        # Check for malicious traffic
        is_malicious = (
            predictions.get('traffic_source') == 'MALICIOUS' or
            predictions.get('operator_type') in ['AUTOMATED', 'HUMAN'] and
            predictions.get('operator_type') != 'BENIGN'
        )

        if is_malicious and max_confidence >= self.config['alert_threshold']:
            alert['threat_detected'] = True
            if max_confidence >= 0.95:
                alert['alert_level'] = 'CRITICAL'
            elif max_confidence >= 0.85:
                alert['alert_level'] = 'HIGH'
            else:
                alert['alert_level'] = 'MEDIUM'

            self.alerts_generated += 1
            self.alert_history.append(alert)

        return alert

    def process_flow_batch(self, flows):
        """Process a batch of network flows"""
        batch_results = []
        batch_start = time.time()

        for flow in flows:
            # Make predictions
            predictions, confidences = self.predict_flow(flow)

            # Generate alert if needed
            alert = self.generate_alert(flow, predictions, confidences)

            # Store result
            result = {
                'timestamp': datetime.now().isoformat(),
                'predictions': predictions,
                'confidences': confidences,
                'alert': alert,
                'processing_time': time.time() - batch_start
            }

            batch_results.append(result)
            self.results_history.append(result)
            self.processed_flows += 1

            # Update statistics
            for task, pred in predictions.items():
                self.statistics[f"{task}_{pred}"] += 1

        return batch_results

    def get_statistics(self):
        """Get current processing statistics"""
        current_time = time.time()
        runtime = current_time - self.start_time

        stats = {
            'runtime_seconds': runtime,
            'processed_flows': self.processed_flows,
            'alerts_generated': self.alerts_generated,
            'flows_per_second': self.processed_flows / runtime if runtime > 0 else 0,
            'queue_size': self.traffic_queue.qsize(),
            'recent_alerts': list(self.alert_history)[-10:],  # Last 10 alerts
            'prediction_distribution': dict(self.statistics)
        }

        return stats

# Initialize the real-time analyzer
if models:
    analyzer = RealTimeTrafficAnalyzer(
        models=models,
        scaler=scaler,
        label_encoders=label_encoders,
        feature_names=feature_names,
        config=REALTIME_CONFIG
    )
    print("✓ Real-time analyzer ready")
else:
    print("  Cannot initialize analyzer - no models loaded")
    analyzer = None


INITIALIZING REAL-TIME ANALYZER
✓ Real-time analyzer initialized
  - Models: ['attack_type', 'traffic_source', 'operator_type']
  - Features: 7
  - Queue size: 1000
✓ Real-time analyzer ready


**Network Traffic Simulator**

This cell defines the `NetworkTrafficSimulator` class, which generates synthetic network traffic flows simulating various attack types (e.g., DDoS, PortScan) and benign traffic. It creates realistic flow patterns based on statistical distributions and initializes the simulator if the analyzer is available.

In [7]:
print("\n" + "="*50)
print("NETWORK TRAFFIC SIMULATION")
print("="*50)

class NetworkTrafficSimulator:
    def __init__(self, feature_names):
        self.feature_names = feature_names
        self.attack_patterns = {
            'BENIGN': self._generate_benign_pattern,
            'DDoS': self._generate_ddos_pattern,
            'PortScan': self._generate_portscan_pattern,
            'Infiltration': self._generate_infiltration_pattern,
            'Bot': self._generate_bot_pattern
        }

    def _generate_benign_pattern(self):
        """Generate benign traffic pattern"""
        return {
            'Flow Duration': np.random.normal(10000, 5000),
            'Total Fwd Packets': np.random.randint(5, 50),
            'Total Backward Packets': np.random.randint(5, 50),
            'Total Length of Fwd Packets': np.random.normal(1500, 500),
            'Total Length of Bwd Packets': np.random.normal(1500, 500),
            'Fwd Packet Length Mean': np.random.normal(100, 30),
            'Bwd Packet Length Mean': np.random.normal(100, 30),
            'Flow Bytes/s': np.random.normal(1000, 300),
            'Flow Packets/s': np.random.normal(10, 3),
            'Flow IAT Mean': np.random.normal(1000, 300)
        }

    def _generate_ddos_pattern(self):
        """Generate DDoS attack pattern"""
        return {
            'Flow Duration': np.random.normal(1000, 200),  # Shorter flows
            'Total Fwd Packets': np.random.randint(100, 1000),  # Many packets
            'Total Backward Packets': np.random.randint(0, 5),   # Few responses
            'Total Length of Fwd Packets': np.random.normal(5000, 1000),
            'Total Length of Bwd Packets': np.random.normal(100, 50),
            'Fwd Packet Length Mean': np.random.normal(50, 20),  # Smaller packets
            'Bwd Packet Length Mean': np.random.normal(60, 20),
            'Flow Bytes/s': np.random.normal(10000, 3000),  # High throughput
            'Flow Packets/s': np.random.normal(100, 30),    # High packet rate
            'Flow IAT Mean': np.random.normal(10, 5)        # Low inter-arrival time
        }

    def _generate_portscan_pattern(self):
        """Generate port scan pattern"""
        return {
            'Flow Duration': np.random.normal(100, 50),     # Very short flows
            'Total Fwd Packets': np.random.randint(1, 5),  # Few packets
            'Total Backward Packets': np.random.randint(0, 2),
            'Total Length of Fwd Packets': np.random.normal(200, 50),
            'Total Length of Bwd Packets': np.random.normal(50, 20),
            'Fwd Packet Length Mean': np.random.normal(40, 10),
            'Bwd Packet Length Mean': np.random.normal(40, 10),
            'Flow Bytes/s': np.random.normal(100, 30),
            'Flow Packets/s': np.random.normal(5, 2),
            'Flow IAT Mean': np.random.normal(50, 20)
        }

    def _generate_infiltration_pattern(self):
        """Generate infiltration pattern"""
        return {
            'Flow Duration': np.random.normal(50000, 10000),  # Long flows
            'Total Fwd Packets': np.random.randint(20, 100),
            'Total Backward Packets': np.random.randint(20, 100),
            'Total Length of Fwd Packets': np.random.normal(3000, 800),
            'Total Length of Bwd Packets': np.random.normal(3000, 800),
            'Fwd Packet Length Mean': np.random.normal(150, 50),
            'Bwd Packet Length Mean': np.random.normal(150, 50),
            'Flow Bytes/s': np.random.normal(500, 150),
            'Flow Packets/s': np.random.normal(2, 1),
            'Flow IAT Mean': np.random.normal(5000, 1000)
        }

    def _generate_bot_pattern(self):
        """Generate bot traffic pattern"""
        return {
            'Flow Duration': np.random.normal(5000, 1000),
            'Total Fwd Packets': np.random.randint(10, 50),
            'Total Backward Packets': np.random.randint(10, 50),
            'Total Length of Fwd Packets': np.random.normal(2000, 500),
            'Total Length of Bwd Packets': np.random.normal(2000, 500),
            'Fwd Packet Length Mean': np.random.normal(80, 25),
            'Bwd Packet Length Mean': np.random.normal(80, 25),
            'Flow Bytes/s': np.random.normal(800, 200),
            'Flow Packets/s': np.random.normal(8, 3),
            'Flow IAT Mean': np.random.normal(200, 50)
        }

    def generate_flow(self, attack_type=None):
        """Generate a single network flow"""
        if attack_type is None:
            # Random attack type with realistic distribution
            attack_type = np.random.choice(
                ['BENIGN', 'DDoS', 'PortScan', 'Infiltration', 'Bot'],
                p=[0.7, 0.1, 0.1, 0.05, 0.05]  # 70% benign traffic
            )

        # Generate base pattern
        flow = self.attack_patterns[attack_type]()

        # Add missing features with default values
        for feature in self.feature_names:
            if feature not in flow:
                flow[feature] = np.random.normal(0, 1)

        # Add metadata
        flow['_true_label'] = attack_type
        flow['_timestamp'] = datetime.now().isoformat()

        return flow

    def generate_batch(self, batch_size=100):
        """Generate a batch of network flows"""
        return [self.generate_flow() for _ in range(batch_size)]

# Initialize traffic simulator
if analyzer:
    simulator = NetworkTrafficSimulator(feature_names)
    print("✓ Network traffic simulator initialized")
else:
    simulator = None


NETWORK TRAFFIC SIMULATION
✓ Network traffic simulator initialized


**Real-Time Evaluation Execution**

This cell runs the real-time evaluation by generating and processing network traffic batches for a specified duration (3 minutes at 20 flows/second). It collects predictions, alerts, and performance metrics, printing progress updates and final statistics.

In [8]:
print("\n" + "="*50)
print("REAL-TIME EVALUATION EXECUTION")
print("="*50)

def run_realtime_evaluation(duration_minutes=5, flows_per_second=10):
    """Run real-time evaluation for specified duration"""
    if not analyzer or not simulator:
        print("  Cannot run evaluation - analyzer or simulator not initialized")
        return None

    print(f"Starting {duration_minutes}-minute real-time evaluation...")
    print(f"Target: {flows_per_second} flows/second")

    start_time = time.time()
    end_time = start_time + (duration_minutes * 60)

    evaluation_results = {
        'flows_processed': 0,
        'alerts_generated': 0,
        'predictions': [],
        'performance_metrics': [],
        'alert_timeline': []
    }

    try:
        while time.time() < end_time:
            batch_start = time.time()

            # Generate traffic batch
            batch_size = max(1, int(flows_per_second * REALTIME_CONFIG['processing_interval']))
            traffic_batch = simulator.generate_batch(batch_size)

            # Process batch
            batch_results = analyzer.process_flow_batch(traffic_batch)

            # Collect results
            for i, (flow, result) in enumerate(zip(traffic_batch, batch_results)):
                evaluation_results['predictions'].append({
                    'true_label': flow.get('_true_label', 'UNKNOWN'),
                    'predicted_labels': result['predictions'],
                    'confidences': result['confidences'],
                    'timestamp': result['timestamp']
                })

                if result['alert']['threat_detected']:
                    evaluation_results['alert_timeline'].append({
                        'timestamp': result['timestamp'],
                        'alert_level': result['alert']['alert_level'],
                        'true_label': flow.get('_true_label', 'UNKNOWN'),
                        'predictions': result['predictions']
                    })

            evaluation_results['flows_processed'] += len(traffic_batch)
            evaluation_results['alerts_generated'] = analyzer.alerts_generated

            # Performance metrics
            batch_time = time.time() - batch_start
            evaluation_results['performance_metrics'].append({
                'timestamp': datetime.now().isoformat(),
                'batch_size': len(traffic_batch),
                'processing_time': batch_time,
                'flows_per_second': len(traffic_batch) / batch_time if batch_time > 0 else 0
            })

            # Progress update
            elapsed = time.time() - start_time
            progress = (elapsed / (duration_minutes * 60)) * 100
            if int(elapsed) % 30 == 0:  # Update every 30 seconds
                stats = analyzer.get_statistics()
                print(f"Progress: {progress:.1f}% | "
                      f"Flows: {stats['processed_flows']} | "
                      f"Alerts: {stats['alerts_generated']} | "
                      f"Rate: {stats['flows_per_second']:.1f} flows/sec")

            # Sleep to maintain target rate
            sleep_time = REALTIME_CONFIG['processing_interval'] - batch_time
            if sleep_time > 0:
                time.sleep(sleep_time)

    except KeyboardInterrupt:
        print("\n  Evaluation interrupted by user")
    except Exception as e:
        print(f"\n  Evaluation error: {e}")

    # Final statistics
    total_time = time.time() - start_time
    final_stats = analyzer.get_statistics()

    evaluation_results['summary'] = {
        'total_duration': total_time,
        'total_flows': final_stats['processed_flows'],
        'total_alerts': final_stats['alerts_generated'],
        'average_throughput': final_stats['flows_per_second'],
        'alert_rate': final_stats['alerts_generated'] / final_stats['processed_flows'] if final_stats['processed_flows'] > 0 else 0
    }

    print(f"\n" + "="*50)
    print("EVALUATION COMPLETED")
    print("="*50)
    print(f"Duration: {total_time:.1f} seconds")
    print(f"Flows processed: {final_stats['processed_flows']:,}")
    print(f"Alerts generated: {final_stats['alerts_generated']:,}")
    print(f"Average throughput: {final_stats['flows_per_second']:.1f} flows/second")
    print(f"Alert rate: {evaluation_results['summary']['alert_rate']:.3f}")

    return evaluation_results

# Run the evaluation
print("Starting real-time evaluation...")
evaluation_results = run_realtime_evaluation(duration_minutes=3, flows_per_second=20)


REAL-TIME EVALUATION EXECUTION
Starting real-time evaluation...
Starting 3-minute real-time evaluation...
Target: 20 flows/second


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | e


EVALUATION COMPLETED
Duration: 186.8 seconds
Flows processed: 500
Alerts generated: 0
Average throughput: 2.7 flows/second
Alert rate: 0.000


[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished


**Results Analysis and Visualization**

This cell analyzes the evaluation results, calculating classification metrics like accuracy, precision, recall, and F1-score for traffic source predictions. It generates visualizations (alert distribution, throughput, confusion matrix, and confidence distribution) and saves results and a detailed report to files.

In [9]:
if evaluation_results:
    print("\n" + "="*50)
    print("RESULTS ANALYSIS")
    print("="*50)

    # Calculate accuracy metrics
    predictions_df = pd.DataFrame(evaluation_results['predictions'])

    if len(predictions_df) > 0:
        # Traffic source accuracy
        if 'traffic_source' in predictions_df['predicted_labels'].iloc[0]:
            true_sources = []
            pred_sources = []

            for _, row in predictions_df.iterrows():
                true_label = row['true_label']
                true_source = 'BENIGN' if true_label == 'BENIGN' else 'MALICIOUS'
                pred_source = row['predicted_labels'].get('traffic_source', 'UNKNOWN')

                true_sources.append(true_source)
                pred_sources.append(pred_source)

            # Calculate metrics
            from sklearn.metrics import accuracy_score, precision_recall_fscore_support

            accuracy = accuracy_score(true_sources, pred_sources)
            precision, recall, f1, _ = precision_recall_fscore_support(
                true_sources, pred_sources, average='weighted', zero_division=0
            )

            print(f"Traffic Source Classification:")
            print(f"  Accuracy: {accuracy:.3f}")
            print(f"  Precision: {precision:.3f}")
            print(f"  Recall: {recall:.3f}")
            print(f"  F1-Score: {f1:.3f}")

        # Create visualizations
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))

        # 1. Alert timeline
        alert_timeline = evaluation_results['alert_timeline']
        if alert_timeline:
            alert_df = pd.DataFrame(alert_timeline)
            alert_counts = alert_df['alert_level'].value_counts()
            alert_counts.plot(kind='bar', ax=axes[0,0], title='Alert Distribution by Level')
            axes[0,0].set_xlabel('Alert Level')
            axes[0,0].set_ylabel('Count')
        else:
            axes[0,0].text(0.5, 0.5, 'No Alerts Generated', ha='center', va='center', transform=axes[0,0].transAxes)
            axes[0,0].set_title('Alert Distribution')

        # 2. Throughput over time
        perf_df = pd.DataFrame(evaluation_results['performance_metrics'])
        if len(perf_df) > 0:
            perf_df['flows_per_second'].plot(ax=axes[0,1], title='Processing Throughput Over Time')
            axes[0,1].set_xlabel('Batch Number')
            axes[0,1].set_ylabel('Flows/Second')

        # 3. True vs Predicted Distribution
        if 'traffic_source' in predictions_df['predicted_labels'].iloc[0]:
            true_pred_df = pd.DataFrame({
                'True': true_sources,
                'Predicted': pred_sources
            })
            confusion = pd.crosstab(true_pred_df['True'], true_pred_df['Predicted'])
            sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', ax=axes[1,0])
            axes[1,0].set_title('True vs Predicted Traffic Source')
            axes[1,0].set_xlabel('Predicted')
            axes[1,0].set_ylabel('True')
        else:
            axes[1,0].text(0.5, 0.5, 'No Traffic Source Predictions',
                          ha='center', va='center', transform=axes[1,0].transAxes)
            axes[1,0].set_title('Traffic Source Confusion Matrix')

        # 4. Alert Confidence Distribution
        if len(predictions_df) > 0 and 'traffic_source' in predictions_df['predicted_labels'].iloc[0]:
            confidences = [row['confidences'].get('traffic_source', 0)
                          for _, row in predictions_df.iterrows()]
            axes[1,1].hist(confidences, bins=20, edgecolor='black')
            axes[1,1].set_title('Prediction Confidence Distribution')
            axes[1,1].set_xlabel('Confidence Score')
            axes[1,1].set_ylabel('Count')
        else:
            axes[1,1].text(0.5, 0.5, 'No Confidence Scores Available',
                          ha='center', va='center', transform=axes[1,1].transAxes)
            axes[1,1].set_title('Confidence Distribution')

        plt.tight_layout()

        # Save visualization
        vis_path = os.path.join(EVALUATION_PATHS['visualizations'], f'evaluation_plots_{timestamp}.png')
        plt.savefig(vis_path)
        plt.close()
        print(f"✓ Visualizations saved to: {vis_path}")

    # Save evaluation results
    results_path = os.path.join(EVALUATION_PATHS['results'], f'evaluation_results_{timestamp}.json')
    with open(results_path, 'w') as f:
        json.dump(evaluation_results, f, indent=2)
    print(f"✓ Results saved to: {results_path}")

    # Generate detailed report
    report_path = os.path.join(EVALUATION_PATHS['reports'], f'evaluation_report_{timestamp}.txt')
    with open(report_path, 'w') as f:
        f.write(f"CICIDS2017 Real-Time Evaluation Report\n")
        f.write(f"{'='*50}\n")
        f.write(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

        f.write("SYSTEM CONFIGURATION\n")
        f.write(f"{'-'*30}\n")
        f.write(f"Batch Size: {REALTIME_CONFIG['batch_size']}\n")
        f.write(f"Alert Threshold: {REALTIME_CONFIG['alert_threshold']}\n")
        f.write(f"Window Size: {REALTIME_CONFIG['window_size']} seconds\n")
        f.write(f"Max Queue Size: {REALTIME_CONFIG['max_queue_size']}\n\n")

        f.write("MODEL INFORMATION\n")
        f.write(f"{'-'*30}\n")
        for task, meta in model_metadata.items():
            f.write(f"{task} Model:\n")
            for key, value in meta.items():
                f.write(f"  {key}: {value}\n")
            f.write("\n")

        f.write("EVALUATION SUMMARY\n")
        f.write(f"{'-'*30}\n")
        for key, value in evaluation_results['summary'].items():
            f.write(f"{key}: {value:.2f}\n")

        if 'traffic_source' in predictions_df['predicted_labels'].iloc[0]:
            f.write("\nTRAFFIC SOURCE CLASSIFICATION METRICS\n")
            f.write(f"{'-'*30}\n")
            f.write(f"Accuracy: {accuracy:.3f}\n")
            f.write(f"Precision: {precision:.3f}\n")
            f.write(f"Recall: {recall:.3f}\n")
            f.write(f"F1-Score: {f1:.3f}\n")

        f.write("\nALERT DISTRIBUTION\n")
        f.write(f"{'-'*30}\n")
        if alert_timeline:
            alert_counts = pd.DataFrame(alert_timeline)['alert_level'].value_counts()
            for level, count in alert_counts.items():
                f.write(f"{level}: {count}\n")

        f.write("\nPREDICTION DISTRIBUTION\n")
        f.write(f"{'-'*30}\n")
        for key, value in analyzer.get_statistics()['prediction_distribution'].items():
            f.write(f"{key}: {value}\n")

    print(f"✓ Report saved to: {report_path}")


RESULTS ANALYSIS
Traffic Source Classification:
  Accuracy: 0.728
  Precision: 0.530
  Recall: 0.728
  F1-Score: 0.613
✓ Visualizations saved to: /content/drive/MyDrive/furssah/evaluation_output_20250529_231829/visualizations/evaluation_plots_20250529_231829.png
✓ Results saved to: /content/drive/MyDrive/furssah/evaluation_output_20250529_231829/results/evaluation_results_20250529_231829.json
✓ Report saved to: /content/drive/MyDrive/furssah/evaluation_output_20250529_231829/reports/evaluation_report_20250529_231829.txt


**System Shutdown**

This cell performs cleanup operations, clearing the traffic queue and saving final statistics to a file. It also clears in-memory data structures and prints the completion timestamp, marking the end of the evaluation process.

In [10]:
print("\n" + "="*50)
print("SYSTEM SHUTDOWN")
print("="*50)

def cleanup():
    """Perform cleanup operations"""
    try:
        # Clear queue
        while not analyzer.traffic_queue.empty():
            analyzer.traffic_queue.get()

        # Save final statistics
        final_stats_path = os.path.join(EVALUATION_PATHS['logs'], f'final_stats_{timestamp}.json')
        with open(final_stats_path, 'w') as f:
            json.dump(analyzer.get_statistics(), f, indent=2)
        print(f"✓ Final statistics saved to: {final_stats_path}")

        # Clear memory
        analyzer.results_history.clear()
        analyzer.alert_history.clear()
        analyzer.statistics.clear()

        print("✓ System cleanup completed")

    except Exception as e:
        print(f"  Cleanup error: {e}")

if analyzer:
    cleanup()

print("\n" + "="*50)
print(f"Evaluation completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*50)


SYSTEM SHUTDOWN
✓ Final statistics saved to: /content/drive/MyDrive/furssah/evaluation_output_20250529_231829/logs/final_stats_20250529_231829.json
✓ System cleanup completed

Evaluation completed at: 2025-05-29 23:21:42
